In [120]:
import sys
from glob import glob

%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import thicket as th

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

Read all files

In [121]:
tk = th.Thicket.from_caliperreader(glob("cali/*/*.cali"))

(2/2) Creating Thicket: 100%|██████████| 1108/1108 [00:17<00:00, 62.08it/s]
c:\Users\Surya\Documents\TAMU\TAMU_Y3S1\CSCE 435\.venv\Lib\site-packages\thicket\ensemble.py:409: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].replace({numerical_fill_value: None}, inplace=True)


View Calltree

In [122]:
print(tk.tree(metric_column="Avg time/rank"))

  _____ _     _      _        _   
 |_   _| |__ (_) ___| | _____| |_ 
   | | | '_ \| |/ __| |/ / _ \ __|
   | | | | | | | (__|   <  __/ |_ 
   |_| |_| |_|_|\___|_|\_\___|\__|  v2024.1.0

1.110 main
├─ 0.001 MPI_Barrier
├─ 0.001 MPI_Bcast
├─ 0.003 MPI_Comm_dup
├─ 0.000 MPI_Finalize
│  ├─ nan MPI_Comm_dup
│  ├─ nan MPI_Finalize
│  ├─ nan MPI_Finalized
│  └─ nan MPI_Initialized
├─ 0.000 MPI_Finalized
├─ 0.000 MPI_Init
│  └─ nan MPI_Init
├─ 0.000 MPI_Initialized
├─ 0.017 MPI_Reduce
├─ 0.001 comm
│  ├─ nan MPI_Barrier
│  │  └─ nan MPI_Barrier
│  ├─ nan comm_large
│  │  ├─ nan MPI_Recv
│  │  ├─ nan MPI_Reduce
│  │  ├─ nan MPI_Scatterv
│  │  │  └─ nan MPI_Scatterv
│  │  └─ nan MPI_Send
│  └─ 0.001 comm_small
│     └─ 0.001 MPI_Gather
├─ 0.039 comp
│  ├─ 0.038 comp_large
│  │  ├─ 0.003 MPI_Gatherv
│  │  └─ nan comp_large
│  │     └─ nan comp
│  │        ├─ nan MPI_Comm_dup
│  │        ├─ nan MPI_Finalize
│  │        ├─ nan MPI_Finalized
│  │        ├─ nan MPI_Initialized
│  │        ├─ nan MPI

In [123]:
extracted_md = tk.metadata[['algorithm', 'programming_model', 'data_type', 'size_of_data_type', 
          'input_size', 'input_type', 'num_procs', 'scalability', 
          'group_num', 'implementation_source']]
print(extracted_md.to_markdown(index=False))

| algorithm   | programming_model   | data_type   |   size_of_data_type |   input_size | input_type       |   num_procs | scalability   |   group_num | implementation_source   |
|:------------|:--------------------|:------------|--------------------:|-------------:|:-----------------|------------:|:--------------|------------:|:------------------------|
| sample      | mpi                 | int         |                   4 |      1048576 | Sorted           |          16 | strong        |           5 | handwritten             |
| radix       | mpi                 | int         |                   4 |       262144 | Reverse Sorted   |           8 | strong        |           5 | handwritten             |
| sample      | mpi                 | int         |                   4 |     67108864 | ReverseSorted    |         256 | strong        |           5 | handwritten             |
| merge       | mpi                 | int         |                   4 |        65536 | 1% Perturbed     |   

Extract `algorithm`, `num_procs`, `input_size`, and `input_type` from metadata and inject into dataframe as columns.

In [124]:
tk.metadata_column_to_perfdata("algorithm")
tk.metadata_column_to_perfdata("num_procs")
tk.metadata_column_to_perfdata("input_size")
tk.metadata_column_to_perfdata("input_type")

tk.dataframe = tk.dataframe.reset_index().set_index(["algorithm", "num_procs", "input_size", "input_type"]).sort_index()

In [125]:
perfdata = tk.dataframe.where(tk.dataframe['name'] == 'comp_large')
perfdata = perfdata.reset_index()
perfdata = perfdata[perfdata['profile'].notna()]
display(perfdata.head())

,algorithm,num_procs,input_size,input_type,node,profile,nid,spot.channel,Min time/rank,Max time/rank,Avg time/rank,Total time,Variance time/rank,Min time/rank (exc),Max time/rank (exc),Avg time/rank (exc),Total time (exc),Calls/rank (min),Calls/rank (avg),Calls/rank (max),Calls/rank (total),name
26,bitonic,2,65536,perturbed,"{'name': 'comp_large', 'type': 'function'}",3.655061e+09,6.0,regionprofile,0.022897,0.028040,0.025468,0.050936,0.000007,0.022897,0.028040,0.025468,0.050936,1.0,1.5,2.0,3.0,comp_large
75,bitonic,2,65536,random,"{'name': 'comp_large', 'type': 'function'}",1.352121e+09,6.0,regionprofile,0.028188,0.033607,0.030898,0.061796,0.000007,0.028188,0.033607,0.030898,0.061796,1.0,1.5,2.0,3.0,comp_large
124,bitonic,2,65536,reverse,"{'name': 'comp_large', 'type': 'function'}",1.488322e+08,6.0,regionprofile,0.022342,0.027976,0.025159,0.050319,0.000008,0.022342,0.027976,0.025159,0.050319,1.0,1.5,2.0,3.0,comp_large
173,bitonic,2,65536,sorted,"{'name': 'comp_large', 'type': 'function'}",8.242827e+08,6.0,regionprofile,0.024519,0.029875,0.027197,0.054394,0.000007,0.024519,0.029875,0.027197,0.054394,1.0,1.5,2.0,3.0,comp_large
222,bitonic,2,262144,perturbed,"{'name': 'comp_large', 'type': 'function'}",3.184030e+08,6.0,regionprofile,0.114094,0.137977,0.126035,0.252071,0.000143,0.114094,0.137977,0.126035,0.252071,1.0,1.5,2.0,3.0,comp_large


In [126]:
tk.dataframe.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 54341 entries, ('bitonic', np.int64(2), np.int64(65536), 'perturbed') to ('sample', np.int64(1024), np.int64(268435456), 'Sorted')
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   node                 54341 non-null  object 
 1   profile              54341 non-null  int64  
 2   nid                  19193 non-null  float64
 3   spot.channel         19193 non-null  object 
 4   Min time/rank        19193 non-null  float64
 5   Max time/rank        19193 non-null  float64
 6   Avg time/rank        19193 non-null  float64
 7   Total time           19193 non-null  float64
 8   Variance time/rank   19193 non-null  float64
 9   Min time/rank (exc)  19193 non-null  float64
 10  Max time/rank (exc)  19193 non-null  float64
 11  Avg time/rank (exc)  19193 non-null  float64
 12  Total time (exc)     19193 non-null  float64
 13  Calls/rank (min)     17802 non-

In [127]:
import os

def save_fig(path : str, filename : str):
    path_dirs = path.split('/')
    for i in range(len(path_dirs)):
        intermediate_path = '/'.join(path_dirs[:i+1])
        if not os.path.exists(intermediate_path):
            os.mkdir(intermediate_path)
    plt.savefig(os.path.join(path, filename))

def plot_rank_times(input_size: int, df_by_size: pd.DataFrame, input_types: list, algorithm: str, show=True, save=False):
    fig, axs = plt.subplots(2, 2, figsize=(12, 10))
    fig.suptitle(f'Impact of Processors on Time / Rank (Input Size = {input_size})')

    for i, input_type in enumerate(input_types):
        df_by_type = df_by_size[df_by_size['input_type'] == input_type]

        num_procs = df_by_type['num_procs']
        min_time = df_by_type['Min time/rank']
        avg_time = df_by_type['Avg time/rank']
        max_time = df_by_type['Max time/rank']

        ax = axs[i // 2, i % 2]
        ax.plot(num_procs, min_time, label='Min time/rank', marker='o')
        ax.plot(num_procs, avg_time, label='Avg time/rank', marker='o')
        ax.plot(num_procs, max_time, label='Max time/rank', marker='o')

        ax.set_title(f'Input Type: {input_type}')
        ax.set_xlabel('Number of Processors')
        ax.set_ylabel('Time (s)')
        ax.legend()

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    if save:
        save_fig(f'plots/{algorithm}', f'{algorithm}_performance_rank_a{input_size}.png')
    if show:
        plt.show()
    else:
        plt.close(fig)

def plot_total_time(input_size: int, df_by_size: pd.DataFrame, input_types: list, algorithm: str, show=True, save=False):
    fig, ax = plt.subplots(figsize=(12, 5))
    ax.set_title(f'Impact of Processors on Total time (Input Size = {input_size})')
    
    for input_type in input_types:
        df_by_type = df_by_size[df_by_size['input_type'] == input_type]
        total_time = df_by_type.groupby('num_procs')['Total time'].mean()
        ax.plot(total_time.index, total_time.values, label=f'Total time ({input_type})', marker='o')

    ax.set_xlabel('Number of Processors')
    ax.set_ylabel('Time (s)')
    ax.legend()
    plt.tight_layout()
    if save:
        save_fig(f'plots/{algorithm}', f'{algorithm}_performance_total_a{input_size}.png')
    if show:
        plt.show()
    else:
        plt.close(fig)

def plot_variance_time(input_size: int, df_by_size: pd.DataFrame, input_types: list, algorithm: str, show=True, save=False):
    fig, ax = plt.subplots(figsize=(12, 5))
    ax.set_title(f'Impact of Processors on Variance Time/Rank (Input Size = {input_size})')
    
    for input_type in input_types:
        df_by_type = df_by_size[df_by_size['input_type'] == input_type]
        variance_time = df_by_type.groupby('num_procs')['Variance time/rank'].mean()
        ax.plot(variance_time.index, variance_time.values, label=f'Variance time ({input_type})', marker='o')

    ax.set_xlabel('Number of Processors')
    ax.set_ylabel('Time (s)')
    ax.legend()
    plt.tight_layout()
    if save:
        save_fig(f'plots/{algorithm}', f'{algorithm}_performance_variance_a{input_size}.png')
    if show:
        plt.show()
    else:
        plt.close(fig)

def plot_process_performance(perfdata : pd.DataFrame, algorithm : str, show=True, save=False):
    bitonic_df = perfdata[perfdata['algorithm'] == algorithm]
    
    input_sizes = bitonic_df['input_size'].unique()
    input_types = bitonic_df['input_type'].unique()

    for input_size in input_sizes:
        df_by_size = bitonic_df[bitonic_df['input_size'] == input_size]
        
        plot_rank_times(input_size, df_by_size, input_types, algorithm, show, save)
        plot_total_time(input_size, df_by_size, input_types, algorithm, show, save)
        plot_variance_time(input_size, df_by_size, input_types, algorithm, show, save)


In [128]:
# plot_process_performance(perfdata, 'bitonic', show=False, save=True)

As we can see from the above plots, the computation time by rank scales down negative exponentially as expected when we scale up the number of processors. This is because the bitonic sort implementation recursively divides the computational load between processors such that each only has to manage an even subarray. 

We also see that as we increase the input size, the exponential speedup by number of processes is maintained, while our total time does increase. This is expected as greater array sizes means increased computational costs per processor, additional memory allocation, and more data that needs to be communicated between processes. 

Another good indication is that the variance in computational time between processes likewise displays an exponential decay as we ramp up the number of processors. This is good news as it indicates this approach is evenly distributing the workload among processors. The key factor in this is the parallel merging algorithm I implemented that utilizes partner processes to hierarchically merge the locally sorted subarrays, trickling up into the master process's final sorted array.

Overall, the scalability of this approach is definitely quite strong as we ramp up the number of processors and exponentiate our input size, but there is definitely still room for improvement on cutting down communication costs and complexities.

In [135]:
def strong_scaling_plot(input_size : int, section : str, thicket_df : pd.DataFrame, algorithm : str, show=True, save=False):
    fig = plt.figure(figsize=(12, 10))
    plt.title(f'Strong Scaling (Algorithm = {algorithm} sort, Input Size = {input_size})')
    
    perfdata = thicket_df[thicket_df['name'] == section]
    perfdata = perfdata[perfdata['profile'].notna()]
    perfdata = perfdata[perfdata['algorithm'] == algorithm]
    perfdata = perfdata[perfdata['input_size'] == input_size]

    input_types = perfdata['input_type'].unique()

    for input_type in input_types:
        df_by_type = perfdata[perfdata['input_type'] == input_type]
        num_procs = df_by_type['num_procs']
        avg_time = df_by_type['Avg time/rank']
        
        plt.plot(num_procs, avg_time, label=input_type, marker='o')

    plt.xlabel('Number of Processors')
    plt.ylabel('Time (s)')
    plt.grid()
    plt.legend()

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])

    if save:
        save_fig(f'plots/{algorithm}', f'{algorithm}_strongscaling_a{input_size}_{section}.png')
    if show:
        plt.show()
    else:
        plt.close(fig)


In [136]:

def strong_scaling_speedup_plot(input_type : str, section : str, thicket_df : pd.DataFrame, algorithm : str, show=True, save=False):
    fig = plt.figure(figsize=(12, 10))
    plt.title(f'Strong Scaling Speedup (Algorithm = {algorithm}, Input Type = {input_type})')
    
    perfdata = thicket_df[thicket_df['name'] == section]
    perfdata = perfdata[perfdata['profile'].notna()]
    perfdata = perfdata[perfdata['algorithm'] == algorithm]
    perfdata = perfdata[perfdata['input_type'] == input_type]

    input_sizes = list(perfdata['input_size'].unique())

    for input_size in input_sizes:
        df_by_size = perfdata[perfdata['input_size'] == input_size]
        num_procs = df_by_size['num_procs']
        t1 = df_by_size[num_procs == 2]['Avg time/rank'].iloc[0] * 2
        speedup = t1 / df_by_size['Avg time/rank']
        
        plt.plot(num_procs, speedup, label=input_size, marker='o')

    plt.xlabel('Number of Processors')
    plt.ylabel('Speedup')
    plt.grid()
    plt.legend()

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])

    if save:
        save_fig(f'plots/{algorithm}', f'{algorithm}_strongscaling_speedup_{input_type}_{section}.png')
    if show:
        plt.show()
    else:
        plt.close(fig)


In [170]:

def weak_scaling_speedup_plot(input_type : str, section : str, thicket_df : pd.DataFrame, algorithm : str, show=True, save=False):
    fig = plt.figure(figsize=(12, 10))
    plt.title(f'Weak Scaling Scaled Speedup (Algorithm = {algorithm} sort, Input Type = {input_type})')
    
    perfdata = thicket_df[thicket_df['name'] == section]
    perfdata = perfdata[perfdata['profile'].notna()]
    perfdata = perfdata[perfdata['algorithm'] == algorithm]

    df_by_type = perfdata[perfdata['input_type'] == input_type]

    weak_procs = sorted([int(el) for el in list(df_by_type['num_procs'].unique())])
    weak_sizes = sorted([int(el) for el in list(df_by_type['input_size'].unique())])
    weak_efficiencies = []

    max_len = min(len(weak_procs), len(weak_sizes))
    base_procs = weak_procs[0]

    for i, n_proc in enumerate(weak_procs[:max_len]):
        df_by_size = df_by_type[df_by_type['input_size'] == weak_sizes[i]]

        base_time = df_by_size[df_by_type['num_procs'] == base_procs]['Avg time/rank'].iloc[0] * base_procs
        avg_time = df_by_size[df_by_type['num_procs'] == n_proc]['Avg time/rank'].iloc[0]
        scaled_efficiency = base_time / avg_time
        weak_efficiencies.append(scaled_efficiency)

    plt.plot(weak_procs[:max_len], weak_efficiencies[:max_len], label=input_type, marker='o')
        
    plt.xlabel('Number of Processors')
    plt.ylabel('Scaled Efficiency')
    plt.grid()

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])

    if save:
        save_fig(f'plots/{algorithm}', f'{algorithm}_weakscaling_speedup_{input_type}_{section}.png')
    if show:
        plt.show()
    else:
        plt.close(fig)


In [171]:
def normalize_column(df : pd.DataFrame, column_name : str, new_values : list[str]):
    for new_val in new_values:
        df[column_name] = df[column_name].apply(
            lambda x: new_val if pd.notna(x) and new_val.lower() in x.lower() else x
        )
    return df

In [172]:
thicket_df = tk.dataframe.reset_index()
thicket_df = normalize_column(thicket_df, column_name='input_type', new_values=['random', 'perturbed', 'reverse', 'sorted'])
input_sizes = list(thicket_df['input_size'].unique())
input_types = list(thicket_df['input_type'].unique())
algorithms = list(thicket_df['algorithm'].unique())

for algorithm in algorithms:
    for section in ['comm', 'comp_large', 'main']:
        for input_size in input_sizes:
            strong_scaling_plot(input_size, section, thicket_df, algorithm, show=False, save=True)
        for input_type in input_types:
            strong_scaling_speedup_plot(input_type, section, thicket_df, algorithm, show=False, save=True)
            weak_scaling_speedup_plot(input_type, section, thicket_df, algorithm, show=False, save=True)

C:\Users\Surya\AppData\Local\Temp\ipykernel_25468\4056141702.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  base_time = df_by_size[df_by_type['num_procs'] == base_procs]['Avg time/rank'].iloc[0] * base_procs
C:\Users\Surya\AppData\Local\Temp\ipykernel_25468\4056141702.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  avg_time = df_by_size[df_by_type['num_procs'] == n_proc]['Avg time/rank'].iloc[0]
C:\Users\Surya\AppData\Local\Temp\ipykernel_25468\4056141702.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  base_time = df_by_size[df_by_type['num_procs'] == base_procs]['Avg time/rank'].iloc[0] * base_procs
C:\Users\Surya\AppData\Local\Temp\ipykernel_25468\4056141702.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  avg_time = df_by_size[df_by_type['num_procs'] == n_proc]['Avg time/rank'].iloc[0]
C:\Users\Surya\AppData\Local\Temp\ipykernel_25468\40

In [173]:
tk_cache = th.Thicket.from_caliperreader(glob("cali_for_cache_misses/*/*.cali"))

(2/2) Creating Thicket: 100%|██████████| 53/53 [00:00<00:00, 774.86it/s]
c:\Users\Surya\Documents\TAMU\TAMU_Y3S1\CSCE 435\.venv\Lib\site-packages\thicket\ensemble.py:409: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].replace({numerical_fill_value: None}, inplace=True)


In [174]:
tk_cache.metadata_column_to_perfdata("algorithm")
tk_cache.metadata_column_to_perfdata("num_procs")
tk_cache.metadata_column_to_perfdata("input_size")
tk_cache.metadata_column_to_perfdata("input_type")

tk_cache.dataframe = tk_cache.dataframe.reset_index().set_index(["algorithm", "num_procs", "input_size", "input_type"]).sort_index()
cache_df = tk_cache.dataframe.reset_index()
cache_df = normalize_column(cache_df, column_name='input_type', new_values=['random', 'perturbed', 'reverse', 'sorted'])

In [175]:
cache_df.head()

,algorithm,num_procs,input_size,input_type,node,profile,nid,spot.channel,Min time/rank,Max time/rank,Avg time/rank,Total time,Min L1 misses/rank (exc),Max L1 misses/rank (exc),Avg L1 misses/rank (exc),Total L1 misses (exc),Min L2 misses/rank (exc),Max L2 misses/rank (exc),Avg L2 misses/rank (exc),Total L2 misses (exc),Min L1 misses/rank,Max L1 misses/rank,Avg L1 misses/rank,Total L1 misses,Min L2 misses/rank,Max L2 misses/rank,Avg L2 misses/rank,Total L2 misses,Variance time/rank,Min time/rank (exc),Max time/rank (exc),Avg time/rank (exc),Total time (exc),Calls/rank (min),Calls/rank (avg),Calls/rank (max),Calls/rank (total),name
0,bitonic,16,67108864,random,"{'name': 'main', 'type': 'function'}",426130800,1.0,regionprofile,23.973296,23.973767,23.973370,383.573915,1002111.0,1023629.0,1.009552e+06,16152833.0,370449.0,385839.0,378360.0625,6053761.0,21046349.0,133333745.0,3.557134e+07,569141511.0,9525576.0,85261312.0,1.902034e+07,304325498.0,0.000000,0.751646,0.757523,0.755440,12.087043,NaN,NaN,NaN,NaN,main
1,bitonic,16,67108864,random,"{'name': 'MPI_Barrier', 'type': 'function'}",426130800,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2,bitonic,16,67108864,random,"{'name': 'MPI_Bcast', 'type': 'function'}",426130800,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3,bitonic,16,67108864,random,"{'name': 'MPI_Comm_dup', 'type': 'function'}",426130800,3.0,regionprofile,0.020715,16.162671,13.902737,222.443795,38836.0,219367.0,1.551532e+05,2482451.0,14457.0,16650.0,15487.5000,247800.0,38836.0,219367.0,1.551532e+05,2482451.0,14457.0,16650.0,1.548750e+04,247800.0,17.497105,0.020715,16.162671,13.902737,222.443795,2.0,2.0,2.0,32.0,MPI_Comm_dup
4,bitonic,16,67108864,random,"{'name': 'MPI_Finalize', 'type': 'function'}",426130800,12.0,regionprofile,0.000010,0.000015,0.000012,0.000185,233.0,284.0,2.638750e+02,4222.0,55.0,79.0,61.5625,985.0,233.0,284.0,2.638750e+02,4222.0,55.0,79.0,6.156250e+01,985.0,0.000000,0.000010,0.000015,0.000012,0.000185,1.0,1.0,1.0,16.0,MPI_Finalize


In [185]:
def plot_cache_misses(cache_df: pd.DataFrame, algorithm: str, section : str, input_type : str, array_exp: list[int], show=True, save=False):
    cache_data = cache_df[cache_df['name'] == section]
    cache_data = cache_data[cache_data['profile'].notna()]
    cache_data = cache_data[cache_data['algorithm'] == algorithm]

    input_sizes = [int(2**exp) for exp in array_exp]

    for level in ['L1', 'L2']:
        fig, axs = plt.subplots(1, 2, figsize=(15, 8))
        fig.suptitle(f'{level} Data Cache Misses per Rank for Algorithm = {algorithm} sort, Type = {input_type}, Region = {section}')

        for i, input_size in enumerate(input_sizes):
            df_by_size = cache_data[cache_data['input_size'] == input_size]
            df_by_size = df_by_size[df_by_size['input_type'] == input_type]

            num_procs = df_by_size['num_procs']
            min_miss = df_by_size[f'Min {level} misses/rank']
            avg_miss = df_by_size[f'Avg {level} misses/rank']
            max_miss = df_by_size[f'Max {level} misses/rank']
            total_miss = df_by_size[f'Total {level} misses']

            ax = axs[i]
            ax.plot(num_procs, min_miss, label=f'Min {level} misses/rank', marker='o')
            ax.plot(num_procs, avg_miss, label=f'Avg {level} misses/rank', marker='o')
            ax.plot(num_procs, max_miss, label=f'Max {level} misses/rank', marker='o')
            ax.plot(num_procs, total_miss, label=f'Total {level} misses', marker='o')

            ax.set_title(f'2^{array_exp[i]} values')
            ax.set_xlabel('Number of Processors')
            ax.set_ylabel('Misses / rank')
            ax.legend()

        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        if save:
            save_fig(f'plots/{algorithm}', f'{algorithm}_cache_misses_{level}_{input_type}_{section}.png')
        if show:
            plt.show()
        else:
            plt.close(fig)

In [186]:
input_sizes = list(cache_df['input_size'].unique())
input_types = list(cache_df['input_type'].unique())
algorithms = list(cache_df['algorithm'].unique())

for algorithm in algorithms:
    for input_type in input_types:
        for section in ['comm', 'comp_large', 'main']:
            plot_cache_misses(cache_df, algorithm, section, input_type, [26, 28], show=False, save=True)